In [ ]:
!git clone https://github.com/TNTWEN/OpenVINO-YOLOV4.git

Cloning into 'OpenVINO-YOLOV4'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (419/419), done.
remote: Total 667 (delta 108), reused 337 (delta 29), pack-reused 205
Receiving objects: 100% (667/667), 48.14 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (219/219), done.


In [ ]:
%pwd

'/content'

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [ ]:
%cd /content/OpenVINO-YOLOV4/yolov4-relu/

/content/OpenVINO-YOLOV4/yolov4-relu


In [ ]:
!ls -l

total 44
drwxr-xr-x 2 root root  4096 Feb 11 10:44 cfg
-rwxr-xr-x 1 root root  1552 Feb 11 10:44 convert_weights_pb.py
-rwxr-xr-x 1 root root 10607 Feb 11 10:44 utils.py
-rwxr-xr-x 1 root root   410 Feb 11 10:44 yolov4.json
-rwxr-xr-x 1 root root 11097 Feb 11 10:44 yolo_v4.py
-rw-r--r-- 1 root root  4537 Feb 11 10:44 yolo_v4_tiny.py


In [ ]:
#upload trained yolov4-tiny-weights under yolov4-relu

In [ ]:
!python convert_weights_pb.py --class_names cfg/coco.names --weights_file /content/OpenVINO-YOLOV4/yolov4-relu/yolov4-tiny.weights --data_format NHWC --tiny

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0211 10:51:54.609406 140010177660800 module_wrapper.py:139] From convert_weights_pb.py:38: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0211 10:51:54.610866 140010177660800 module_wrapper.py:139] From convert_weights_pb.py:40: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0211 10:51:54.614436 140010177660800 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.

Convert yolov4.pb file to openvino

In [ ]:
%cd /content/
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## downnload installation files
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz
path = "l_openvino_toolkit_p_2020.1.023.tgz"
# path = "/content/software/Intel OpenVINO 2019 R3.1/l_openvino_toolkit_p_2019.3.376.tgz"
## install openvino
!tar xf "{path}"
%cd l_openvino_toolkit_p_2020.1.023/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight

In [ ]:
%cd l_openvino_toolkit_p_2020.1.023/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

[Errno 2] No such file or directory: 'l_openvino_toolkit_p_2020.1.023/'
/content/OpenVINO-YOLOV4/yolov4-relu
/bin/bash: ./install_openvino_dependencies.sh: No such file or directory


In [ ]:
!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model /content/OpenVINO-YOLOV4/yolov4-relu/frozen_darknet_yolov4_model.pb \
--tensorflow_use_custom_operations_config /content/OpenVINO-YOLOV4/yolo_v4_tiny.json \
--batch 1 \
--data_type FP16 \
--reverse_input_channel \
--output_dir /content/custom_yolov3_tiny/TinyIR

[setupvars.sh] OpenVINO environment initialized
[ WARNING ]  Use of deprecated cli option --tensorflow_use_custom_operations_config detected. Option use in the following releases will be fatal. Please use --transformations_config cli option instead
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/OpenVINO-YOLOV4/yolov4-relu/frozen_darknet_yolov4_model.pb
	- Path for generated IR: 	/content/custom_yolov3_tiny/TinyIR
	- IR output name: 	frozen_darknet_yolov4_model
	- Log level: 	ERROR
	- Batch: 	1
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	True
TensorFlow speci

In [ ]:
pwd

'/content'

In [ ]:
#CHOOSE the directory where you would like to save the blob.
# I use the same --output_dir as above for the IR conversion
blob_dir = "/content/custom_yolov3_tiny/TinyIR/"

#Copy the path of your .xml and .bin files. For that, you can look at the IR
#conversion output cell, select and copy from:
#[SUCCESS] XML file and bin file paths.
#Or you can choose to compile other .xml .bin files from a different location
#
binfile = "/content/custom_yolov3_tiny/TinyIR/frozen_darknet_yolov4_model.bin"
xmlfile = "/content/custom_yolov3_tiny/TinyIR/frozen_darknet_yolov4_model.xml"

import requests

#For openvino 20.01 use this link to compile the blob
url = "http://69.164.214.171:8083/compile"


#open vino 20.02 link:
# url = "69.164.214.171:8081"


#Jacob's notes to self
#OR 7 
#depthai shaves 14, cmx 14
#in depthai.py change if default blob to match 14 14 2 


payload = {'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4'}
files = [
  ('definition', open(xmlfile,'rb')),
  ('weights', open(binfile,'rb'))
]
headers = {
   'Content-Type': 'application/json'
 }
response = requests.post(url, data = payload, files = files)
print(response)
print(response.content)
blobnameraw = response.headers.get('Content-Disposition')
print(blobnameraw)
blobname = blobnameraw[blobnameraw.find('='):][1:]
print(blobname)
with open(blob_dir + blobname, 'wb') as f:
  print('printing...' + blob_dir + blobname)
  f.write(response.content)

<Response [400]>
b'Unknown compile type: '
None


AttributeError: ignored

In [ ]:
# The API dont work, therefore use below links and follow the instructions to convert our model to .blob
# https://docs.luxonis.com/en/latest/pages/model_conversion/#using-online-converter
# http://blobconverter.luxonis.com/